In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds

from surprise import KNNWithMeans, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split

In [2]:
df_movies = pd.read_csv("Data/movies.csv")
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
df_ratings = pd.read_csv("Data/ratings.csv")
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
df_ratings.drop(columns='timestamp', inplace=True)

In [5]:
df_cleaned = df_movies.merge(df_ratings, on='movieId')

In [6]:
df_cleaned.head()

,movieId,title,genres,userId,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5


In [7]:
#popular = df_cleaned.groupby(['title']).agg({"rating":"count"})['rating'].sort_values(ascending=False)

#df_cleaned['Number of Users Watched'] = popular.values


In [13]:
# Count the number of ratings and users for each movie
popular = df_cleaned.groupby(['title']).agg({"rating":"count", "userId":"nunique"})
popular = popular.rename(columns={"userId": "Number of Users"})

# Merge popular with df_cleaned on the 'title' column
df_merged = pd.merge(df_cleaned, popular, on='title', how='inner')


SyntaxError: invalid syntax (<ipython-input-13-e9cf8079c690>, line 2)

In [9]:
popular.head()

,rating,Number of Users
title,,
'71 (2014),1,1
'Hellboy': The Seeds of Creation (2004),1,1
'Round Midnight (1986),2,2
'Salem's Lot (2004),1,1
'Til There Was You (1997),2,2


In [10]:
df_merged.head()

,movieId,title,genres,userId,rating_x,rating_y,Number of Users
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,215,215
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,215,215
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,215,215
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,215,215
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,215,215


In [12]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movieId          100836 non-null  int64  
 1   title            100836 non-null  object 
 2   genres           100836 non-null  object 
 3   userId           100836 non-null  int64  
 4   rating_x         100836 non-null  float64
 5   rating_y         100836 non-null  int64  
 6   Number of Users  100836 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 6.2+ MB


In [11]:
popular = popular.to_frame()
popular.reset_index(level=0, inplace=True)
popular.columns = ['title', 'Number of Users watched']

AttributeError: 'DataFrame' object has no attribute 'to_frame'

In [ ]:
popular.head()

In [ ]:
# Join the "popular" DataFrame with the "df_cleaned" DataFrame
popularity = df_cleaned.groupby('title').agg({'rating': 'count'}).sort_values('rating', ascending=False)
popularity = popularity.rename(columns={'rating': 'Number of Users watched'})
df_clean = popular.merge(popularity[['Number of Users watched']], on='title')


In [ ]:
# Count the number of unique users for each title
num_users_watched = df_cleaned.groupby('title')['userId'].nunique()
popular['Number of Users watched'] = num_users_watched[popular['title']].values

In [ ]:
num_users_watched

In [ ]:
df_clean.head()

In [ ]:
df_clean = df_cleaned[['userId','title','rating']]

In [ ]:
df_clean.head()

In [ ]:
#########################

PLOTS BELOW

In [ ]:
rating_count_df = pd.DataFrame(df_clean.groupby(['rating']).size(), columns=['Num Ratings'])
rating_count_df



PLOTS ABOVE

In [ ]:
rsagfwas

In [ ]:
import surprise
from surprise import Dataset, Reader, SVD
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Load the dataset using Surprise
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(df_clean[['userId', 'title', 'rating']], reader)

In [ ]:
train, test = train_test_split(data, test_size=0.2, random_state=42)


In [ ]:
from surprise import SVD

algo = SVD()
trainset = data.build_full_trainset()
algo.fit(trainset)

In [ ]:
#algo.test(test)

In [ ]:
surprise.accuracy.mae(algo.test(test))

In [ ]:
surprise.accuracy.rmse(algo.test(test))

In [ ]:
from surprise.prediction_algorithms.matrix_factorization import SVD
SVD = SVD()

In [ ]:
from surprise.model_selection import RandomizedSearchCV

# Define the search space for hyperparameters
param_distributions = {'n_factors': [50, 100, 200],
                       'n_epochs': [10, 20, 30],
                       'lr_all': [0.002, 0.005, 0.01],
                       'reg_all': [0.02, 0.1, 0.4]}
# Create the randomized search object
rs = RandomizedSearchCV(SVD, param_distributions=, n_iter=10, measures=['rmse', 'mae'], cv=5)

# Run the randomized search
rs.fit(data)

# Get the best RMSE score and the corresponding hyperparameters
#best_rmse = rs.best_score['rmse']
#best_params = rs.best_params['rmse']


In [ ]:
print('Best RMSE: ' + str(best_rmse))

In [ ]:
print('Best Params: ' + str(best_params))